In [ ]:
import os
import numpy as np, cv2
import random
import shutil
# Add slash after each location
IM_LOC = '...' # Location of all images
SMALL_LOC = '...' # Location to store super low res images that are compared instead of large images to increase speed
RESOLUTION = 20 # How many images in the X-axis
SMALL_SIZE = (8, 8) # size of super low res images

Run the last cell if it's your first time using a set of images

In [ ]:
IMAGE_NAME = 'test.jpg'
SCALE = 0.5 # Resolution of collage / resolution of original image
original = cv2.resize(cv2.imread(IMAGE_NAME), None, None, SCALE, SCALE, interpolation = cv2.INTER_LANCZOS4)
# Sizes of small images in each axis
x_sq = original.shape[1] // RESOLUTION
y_sq = round(original.shape[0] / RESOLUTION * original.shape[1] / original.shape[0])

canvas = np.zeros((original.shape[0] // y_sq * y_sq, x_sq * RESOLUTION, 3))

In [ ]:
# Storing super low res images in 'data'
im_list = os.listdir(IM_LOC)
if 'desktop.ini' in im_list:
    im_list.remove('desktop.ini')

names = dict(zip(im_list.copy(), [0 for _ in range(len(im_list))])) # Stores how many times an image is used

data = np.zeros((len(im_list), *SMALL_SIZE, 3))
for i, file in enumerate(im_list):
    data[i] = cv2.imread(SMALL_LOC + file)
    

In [ ]:
show = True # Show the process of making the image. Slightly slower if True
MAX_USAGE = 10 # Max number of times an image can be used, lower number results in lower color accuracy and duller image. Higher results in big blocks of the same image which can be ugly

if show:
    cv2.imshow('orig', cv2.resize(original, None, None, 0.25, 0.25))

brk = False # When true, program stops

for x in range(0, original.shape[1] - x_sq, x_sq):
    for y in range(0, original.shape[0] - y_sq, y_sq):
        # Finding closest match
        square = cv2.resize(original[y:y+y_sq, x:x+x_sq], SMALL_SIZE, interpolation = cv2.INTER_LANCZOS4) # Slightly faster if using interpolation that isnt LANCZOS4
        index = np.argmin(np.square(data - square).sum((1,2,3))) # Can use np.absolute instead of np.square
        canvas[y:y+y_sq, x:x+x_sq] = cv2.resize(cv2.imread(IM_LOC + im_list[index]), (x_sq, y_sq))
        
        names[im_list[index]] += 1
        if names[im_list[index]] >= MAX_USAGE:
            im_list.pop(index)
            data = np.reshape(np.delete(data, index, 0), (-1, *data.shape[1:]))
            
        if show:
            cv2.imshow('can', cv2.resize(canvas.astype(np.uint8), None, None, 0.25, 0.25))
            if cv2.waitKey(1)&0xff == ord('q'): # Break when pressing Q
                brk = True
                break
        if brk:
            break
    if brk:
        break
if show:
    cv2.destroyAllWindows()

In [ ]:
# Shows produced image and saves it
cv2.imshow('image', cv2.resize(canvas.astype(np.uint8), None, None, 0.25, 0.25, interpolation = cv2.INTER_LANCZOS4))
cv2.imwrite('result.png', canvas)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:
# Only run the first time using a set of images
# Stores super low res of the set of images
try:
    shutil.rmtree(SMALL_LOC)
except FileNotFoundError as e:
    print(e)
    
os.mkdir(SMALL_LOC)
for file in os.listdir(IM_LOC):
    try:
        im = cv2.resize(cv2.imread(IM_LOC + file), SMALL_SIZE, interpolation = cv2.INTER_LANCZOS4)
        cv2.imwrite(SMALL_LOC + file, im)
    except Exception as e:
        print(e)

[WinError 3] The system cannot find the path specified: 'small_flowers/'
